In [ ]:
TE = joinpath(tempdir(), "GSEA.test")

if isdir(TE)

    rm(TE; recursive = true)

    println("Removed ", TE, ".")

end

mkdir(TE)

println("Made ", TE, ".")

In [2]:
#using Revise

---

In [3]:
using OnePiece

using GSEA

┌ Info: Precompiling GSEA [f74a9ba8-6f1f-48f2-98b6-a79c46cc06f7]
└ @ Base loading.jl:1423
ERROR: LoadError: ArgumentError: can't repeat a string -51 times
Stacktrace:
  [1] repeat(s::String, r::Int64)
    @ Base ./strings/substring.jl:226
  [2] ^
    @ ./strings/basic.jl:721 [inlined]
  [3] tab
    @ ~/.julia/packages/Comonicon/ryWxf/src/ast/printing.jl:1 [inlined]
  [4] print_indent_content(io::IOContext{IOBuffer}, text::String, t::Comonicon.AST.Terminal, firstline::Int64)
    @ Comonicon.AST ~/.julia/packages/Comonicon/ryWxf/src/ast/printing.jl:234
  [5] print_sig_brief(io::IOContext{IOBuffer}, cmd::Comonicon.AST.LeafCommand, t::Comonicon.AST.Terminal)
    @ Comonicon.AST ~/.julia/packages/Comonicon/ryWxf/src/ast/printing.jl:226
  [6] print_body(io::IOContext{IOBuffer}, cmd::Comonicon.AST.NodeCommand, t::Comonicon.AST.Terminal)
    @ Comonicon.AST ~/.julia/packages/Comonicon/ryWxf/src/ast/printing.jl:170
  [7] print_cmd(io::IOContext{IOBuffer}, cmd::Comonicon.AST.NodeCommand, t::Como

LoadError: Failed to precompile GSEA [f74a9ba8-6f1f-48f2-98b6-a79c46cc06f7] to /Users/kwat/.julia/compiled/v1.7/GSEA/jl_0Ypevf.

In [ ]:
da = joinpath(@__DIR__, "data")

se = joinpath(da, "h.all.v7.1.symbols.gmt")

;

## select_set

In [ ]:
GSEA.select_set(OnePiece.io.gmt.read(se), 33, 36)

## run_single_sample_gsea

In [ ]:
ou = joinpath(TE, "single_sample_gsea.tsv")

GSEA.run_single_sample_gsea(
    joinpath(da, "setting", "single_sample_gsea.json"),
    se,
    joinpath(da, "nmf_k9_w.tsv"),
    ou,
)

readdir(TE)

## run_pre_rank_gsea

In [ ]:
ou = joinpath(TE, "pre_rank_gsea")

mkdir(ou)

GSEA.run_pre_rank_gsea(
    joinpath(da, "setting", "pre_rank_gsea.json"),
    se,
    joinpath(da, "gene_score.tsv"),
    ou,
)

readdir(ou)

## run_standard_gsea

In [ ]:
ou = joinpath(TE, "standard_gsea")

mkdir(ou)

GSEA.run_standard_gsea(
    joinpath(da, "setting", "standard_gsea.json"),
    se,
    joinpath(da, "sample_value.tsv"),
    joinpath(da, "nmf_k9_w.tsv"),
    ou,
)

readdir(ou)

## convert_gct_and_cls

In [ ]:
di = joinpath(da, "sarcopenia")

gc = joinpath(
    di,
    "gse111016_allsamplescounts_htseqcov1_sss_forgeo.sarcopenia.vs.normal_counts_collapsed_to_symbols.gct",
)

cl = joinpath(di, "sarcopenia_binary.cls")

GSEA.convert_gct_and_cls(gc, cl, TE)

readdir(di)

## convert_gmt

In [ ]:
gm = joinpath(di, "c2.cp.wikipathways.v7.4.symbols.gmt")

js = joinpath(TE, "set_to_genes.json")

GSEA.convert_gmt(gm, js)

readdir(di)

---

In [ ]:
if isdir(TE)

    rm(TE; recursive = true)

    println("Removed ", TE, ".")

end